In [ ]:
!git clone https://github.com/nguyenthienhy/F5-TTS-Vietnamese.git
!cd F5-TTS-Vietnamese && pip install -e .

In [ ]:
!pip install -q jiwer openai-whisper

In [6]:
import argparse
import codecs
import os
import re
from datetime import datetime
from importlib.resources import files
from pathlib import Path
import gc
import torch
import pandas as pd
import glob

import numpy as np
import soundfile as sf
import tomli
from cached_path import cached_path
from omegaconf import OmegaConf

import jiwer
import whisper

from f5_tts.infer.utils_infer import (
    mel_spec_type,
    target_rms,
    cross_fade_duration,
    nfe_step,
    cfg_strength,
    sway_sampling_coef,
    speed,
    fix_duration,
    infer_process,
    load_model,
    load_vocoder,
    preprocess_ref_audio_text,
    remove_silence_for_generated_wav,
)
from f5_tts.model import DiT, UNetT  # noqa: F401. used for config

In [7]:
# ==========================================
# 1. CẤU HÌNH NGƯỜI DÙNG
# ==========================================

# Model Gốc
BASE_CONFIG = {
    "name": "Base_Model",
    "model_name": "F5TTS_Base",
    "ckpt_file": None, # None = Tự tải
    "vocab_file": ""
}

# Model Finetuned
FINETUNE_CONFIG = {
    "name": "Finetuned_Model",
    "model_name": "F5TTS_Base",
    "ckpt_file": "/kaggle/input/finetuned-f5tts/model_last.pt", # <--- SỬA ĐƯỜNG DẪN Ở ĐÂY
    "vocab_file": "/kaggle/input/finetuned-f5tts/vocab.txt",        # <--- SỬA NẾU CÓ
}

# Load test cases from ground-truth folder
txt_files = sorted(glob.glob("/kaggle/input/truth-ground/ground-truth/*.txt"))
TEST_CASES = []
for file in txt_files:
    with open(file, 'r', encoding='utf-8') as fp:
        text = fp.read().strip()
    TEST_CASES.append(text)

TEST_CASES = TEST_CASES[:]

VOCODER_NAME = "vocos"
REF_AUDIO = "/kaggle/input/male-vivos/data/test/VIVOSDEV01_R002.wav" 
REF_TEXT = "tiếng cạch khựng lại của những khớp sắt"
OUTPUT_DIR = "wer_evaluation_results"

In [8]:
# ==========================================
# 2. HÀM INFERENCE
# ==========================================

def cleanup_memory():
    """Dọn dẹp VRAM triệt để"""
    gc.collect()
    torch.cuda.empty_cache()
    try:
        torch.cuda.ipc_collect()
    except:
        pass

def run_inference_sequence(config, tasks):
    print(f"\n=============================================")
    print(f"STARTING GENERATION FOR: {config['name']}")
    print(f"=============================================")
    
    # 1. Load Vocoder
    print("-> Loading Vocoder...")
    vocoder = load_vocoder(vocoder_name=VOCODER_NAME, is_local=False)
    
    # 2. Config & Model Class
    model_cfg_path = str(files("f5_tts").joinpath(f"configs/{config['model_name']}.yaml"))
    model_cfg = OmegaConf.load(model_cfg_path).model
    
    try:
        model_cls = globals()[model_cfg.backbone]
    except:
        from f5_tts.model import DiT, UNetT
        model_cls = locals()[model_cfg.backbone]

    # 3. Checkpoint Path
    ckpt_file = config['ckpt_file']
    if not ckpt_file:
        repo_name = "F5-TTS"
        ckpt_step = 1200000
        ckpt_file = str(cached_path(f"hf://SWivid/{repo_name}/{config['model_name']}/model_{ckpt_step}.safetensors"))

    # 4. Load Model
    print(f"-> Loading Model from: {ckpt_file}")
    try:
        model = load_model(model_cls, model_cfg.arch, ckpt_file, mel_spec_type=VOCODER_NAME, vocab_file=config['vocab_file'])
    except Exception as e:
        print(f"ERROR loading model {config['name']}: {e}")
        return

    # 5. Load Whisper model (small)
    print("-> Loading Whisper model (small)...")
    whisper_model = whisper.load_model("small")

    # 6. Preprocess Reference
    if "infer/examples/" in REF_AUDIO:
        ref_audio_path = str(files("f5_tts").joinpath(f"{REF_AUDIO}"))
    else:
        ref_audio_path = REF_AUDIO
    
    proc_ref_audio, proc_ref_text = preprocess_ref_audio_text(ref_audio_path, REF_TEXT)
    
    # 7. Generate Loop (Audio + Transcription)
    audio_dir = os.path.join(OUTPUT_DIR, config['name'], "audio")
    text_dir = os.path.join(OUTPUT_DIR, config['name'], "text")
    os.makedirs(audio_dir, exist_ok=True)
    os.makedirs(text_dir, exist_ok=True)
    
    print(f"-> Saving audio to: {audio_dir}")
    print(f"-> Saving transcripts to: {text_dir}")
    
    for i, text in enumerate(tasks):
        print(f"  > Generating sentence {i+1}/{len(tasks)}...")
        try:
            # Generate audio
            audio, sr, _ = infer_process(
                proc_ref_audio, proc_ref_text, text,
                model, vocoder, mel_spec_type=VOCODER_NAME,
                target_rms=0.1, cross_fade_duration=0.15, nfe_step=32, 
                cfg_strength=2.0, sway_sampling_coef=-1.0, speed=1.0
            )
            
            # Save audio file
            audio_filename = f"sample_{i}.wav"
            audio_filepath = os.path.join(audio_dir, audio_filename)
            sf.write(audio_filepath, audio, sr)
            print(f"    Saved audio: {audio_filename}")
            
            # Transcribe with Whisper
            result = whisper_model.transcribe(
                audio_filepath,
                language="vi",
                fp16=torch.cuda.is_available()
            )
            transcript = result["text"].strip()
            
            # Save transcript
            text_filename = f"sample_{i}.txt"
            text_filepath = os.path.join(text_dir, text_filename)
            with open(text_filepath, 'w', encoding='utf-8') as f:
                f.write(transcript)
            print(f"    Saved transcript: {text_filename}")
            
        except Exception as e:
            print(f"    Failed to process sentence {i}: {e}")

    # 8. Cleanup (Quan trọng)
    print("-> Unloading models and cleaning memory...")
    del model
    del vocoder
    del whisper_model
    cleanup_memory()
    print(f"--- Finished {config['name']} ---\n")

In [9]:
# ==========================================
# 4. MAIN
# ==========================================

def main():
    # Base Model
    run_inference_sequence(BASE_CONFIG, TEST_CASES)
    
    # Finetune Model
    if FINETUNE_CONFIG['ckpt_file'] and os.path.exists(FINETUNE_CONFIG['ckpt_file']):
        run_inference_sequence(FINETUNE_CONFIG, TEST_CASES)
    elif FINETUNE_CONFIG['ckpt_file'] is None:
         print("Skipping Finetune model (ckpt_file is None).")
    else:
        print(f"Warning: Checkpoint file not found: {FINETUNE_CONFIG['ckpt_file']}")
        print("Skipping Finetune inference.")

    print("\n=============================================")
    print("ALL INFERENCE DONE.")
    print(f"Files are saved in: {os.path.abspath(OUTPUT_DIR)}")

In [10]:
if __name__ == "__main__":
    main()


STARTING GENERATION FOR: Base_Model
-> Loading Vocoder...
Download Vocos from huggingface charactr/vocos-mel-24khz


pytorch_model.bin:   0%|          | 0.00/54.4M [00:00<?, ?B/s]

F5TTS_Base/model_1200000.safetensors:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

-> Loading Model from: /root/.cache/huggingface/hub/models--SWivid--F5-TTS/snapshots/84e5a410d9cead4de2f847e7c9369a6440bdfaca/F5TTS_Base/model_1200000.safetensors

vocab :  /kaggle/working/F5-TTS-Vietnamese/src/f5_tts/infer/examples/vocab.txt
token :  custom
model :  /root/.cache/huggingface/hub/models--SWivid--F5-TTS/snapshots/84e5a410d9cead4de2f847e7c9369a6440bdfaca/F5TTS_Base/model_1200000.safetensors 

-> Loading Whisper model (small)...



  0%|                                               | 0.00/461M [00:00<?, ?iB/s]
  1%|▌                                     | 6.62M/461M [00:00<00:06, 69.4MiB/s]
  3%|█▏                                    | 14.7M/461M [00:00<00:05, 78.3MiB/s]
  5%|█▊                                    | 22.2M/461M [00:00<00:06, 70.9MiB/s]
  6%|██▍                                   | 29.0M/461M [00:00<00:06, 68.2MiB/s]
  8%|███▏                                  | 38.3M/461M [00:00<00:05, 77.8MiB/s]
 10%|███▊                                  | 46.7M/461M [00:00<00:05, 81.2MiB/s]
 12%|████▍                                 | 54.5M/461M [00:00<00:05, 81.4MiB/s]
 14%|█████▎                                | 63.9M/461M [00:00<00:04, 86.5MiB/s]
 16%|██████                                | 73.9M/461M [00:00<00:04, 91.7MiB/s]
 18%|███████                               | 85.2M/461M [00:01<00:03, 99.9MiB/s]
 21%|███████▊                              | 94.8M/461M [00:01<00:04, 89.3MiB/s]
 22%|████████▊             

...

    Saved audio: sample_998.wav


/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r

    Saved transcript: sample_998.txt
  > Generating sentence 1000/1000...
gen_text 0 thủ tục lên máy bay vào lúc hai giờ ba mươi


Generating audio in 1 batches...



100%|██████████| 1/1 [00:01<00:00,  1.86s/it]


    Saved audio: sample_999.wav
    Saved transcript: sample_999.txt
-> Unloading models and cleaning memory...
--- Finished Finetuned_Model ---


ALL INFERENCE DONE.
Files are saved in: /kaggle/working/wer_evaluation_results


In [13]:
import shutil
from IPython.display import FileLink

# INPUT: đường dẫn folder
folder1_path = "/kaggle/working/wer_evaluation_results/Base_Model/text"   # folder gốc f5tts-base
folder2_path = "/kaggle/working/wer_evaluation_results/Finetuned_Model/text"   # folder gốc f5tts-ours

# zip folder
shutil.make_archive("f5tts-base", 'zip', folder1_path)
shutil.make_archive("f5tts-ours", 'zip', folder2_path)

# hiển thị link download
print("Download f5tts-base.zip:")
display(FileLink("f5tts-base.zip"))

print("Download f5tts-ours.zip:")
display(FileLink("f5tts-ours.zip"))


Download f5tts-base.zip:


/kaggle/working/f5tts-base.zip

Download f5tts-ours.zip:


/kaggle/working/f5tts-ours.zip